In [1]:
cd ../..


/home/nazif/thesis/mirscribe-vcf


/home/nazif/.cache/pypoetry/virtualenvs/mirscribe-vcf-8VJm4tPg-py3.10/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import pandas as pd

from scripts.pipeline_steps import step5 as s5
from scripts.pyensembl_operations import import_pyensembl
g37 = import_pyensembl(37)

from scripts.notebook_operations import setup_notebook

engine, g37, transcripts, genes, mirnas = setup_notebook()

INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.pep.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.pep.all.fa.gz.pickle


In [3]:
df = s5.crawl_and_import_results_into_df(folder_path='results/synth2/', ending_string='a+', assembly=g37)


In [4]:
df.head()

,id,wt_prediction,mut_prediction,pred_difference,vcf_id,mirna_accession,gene_id,mutation_context,mutsig,is_intron,is_gain,is_gene_upregulated
0,PD4116a_X_102396637_A_T_MIMAT0015086,0.474351,0.839948,0.366,PD4116a,MIMAT0015086,NaN,G[T>A]G,SBS3,False,True,False
1,PD4116a_X_103292828_A_T_MIMAT0019199,0.352861,0.804976,0.452,PD4116a,MIMAT0019199,ENSG00000101812,C[T>A]C,SBS3,True,True,False
2,PD4116a_X_103886035_T_A_MIMAT0000763,0.665181,0.287550,-0.378,PD4116a,MIMAT0000763,ENSG00000189108,C[T>A]T,SBS8,True,False,True
3,PD4116a_X_103886035_T_A_MIMAT0022278,0.515179,0.272348,-0.243,PD4116a,MIMAT0022278,ENSG00000189108,C[T>A]T,SBS8,True,False,True
4,PD4116a_X_105078564_T_C_MIMAT0000066,0.798631,0.355989,-0.443,PD4116a,MIMAT0000066,ENSG00000123572,T[T>C]T,SBS3,True,False,True


In [5]:
df = df.merge(genes, on="gene_id", how="left")
df = df.merge(mirnas, on="mirna_accession", how="left")
df.drop(columns=["chr", "start", "end", "sequence", "conservation", "mirna_name_cmc"], inplace=True)

# fill NAs
df["mutsig"] = df["mutsig"].fillna("not_found")
df["gene_id"] = df["gene_id"].fillna("not_found")
df["gene_name"] = df["gene_name"].fillna("not_found")
df["gene_biotype"] = df["gene_biotype"].fillna("not_found")
df["gene_description"] = df["gene_description"].fillna("not_found")
df["cancer_gene_role"] = df["cancer_gene_role"].fillna("neither")
df["tier_cosmic"] = df["tier_cosmic"].fillna(0)

df = df.fillna(False)

df.to_parquet("data/synth2.parquet")

/tmp/ipykernel_41980/3801331413.py:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
